## 1.Import Library

In [100]:
import pandas as pd
import string
import re
from pyvi import ViTokenizer

## 2.Explore Data Analysis

In [101]:
df_train = pd.read_csv('../data/data_origin/Trainfull.csv')
df_test=pd.read_csv('../data/data_origin/Test.csv')

In [102]:
df_train.columns

Index(['index', 'comment', 'n_star', 'date_time', 'label'], dtype='object')

In [103]:
df_train.head(5)

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


In [104]:
df_train.shape

(8898, 5)

### 2.1.Overview of phone dataset

> The dataset consists of 11,122 comments, including of four features: 

>comment: Commentary content.
 n_star: The user evaluates the smartphone's star.

>data_time: The date and time the comment was posted.

>label: Label of comment.

>All samples are in text format. No tokenization has been applied. Users of this dataset are free to use whatever sentence representation they choose.

### 2.2 Structure of dataset

In [105]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8898 entries, 0 to 8897
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      8898 non-null   int64 
 1   comment    8898 non-null   object
 2   n_star     8898 non-null   int64 
 3   date_time  8898 non-null   object
 4   label      8898 non-null   object
dtypes: int64(2), object(3)
memory usage: 347.7+ KB


### 2.3 Summary of statistic

In [106]:
df_train.describe()

,index,n_star
count,8898.000000,8898.000000
mean,4448.500000,3.706900
std,2568.775681,1.505585
min,0.000000,1.000000
25%,2224.250000,3.000000
50%,4448.500000,4.000000
75%,6672.750000,5.000000
max,8897.000000,5.000000


### 2.4. Preprocessing

In [107]:
print(df_train['comment'].values[95:98])

['Điện thoại khá tốt pin trâu khá mượt bắt wifi cực tốt chỉ là thỉnh thoảng máy bị đơ phải thoát ra vào lại và máy ko cập nhật lên miul 12'
 'Lúc trước nghe bảo điện thoại chơi game nóng, nhưng mua về chiến liên quân 3 4h liên tục thì chỉ thấy ấm không nóng như lời đồn, những thứ khác thì vũng chẳng có gì để nói vì quá ngon rồi'
 '1. Máy nếu không chơi game, lướt web..., ít sử dụng thì giữ Pin cao nhất được khoảng 1 ngày 16 tiếng ( Tính luôn cả lúc ngủ nhé vì khi ngủ thì chẳng động gì tới ) . Không được 2 ngày như mn nói đâu \n2. Nhạc nghe cứ cà giật cà giật rất khó chịu luôn\n3. Quay video bị mờ chán \nCòn lại okie 🙆']


> Handle punctuation, handle whitespace, handle icons in strings

In [108]:
def remove_punctuation(comment):
  # Create a translation table
  translator = str.maketrans('', '', string.punctuation)
  # Remove punctuation
  new_string = comment.translate(translator)
  # Remove redudant space and break sign
  new_string = re.sub('[\n ]+', ' ', new_string)
  # Remove emoji icon
  emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags=re.UNICODE)
  new_string = re.sub(emoji_pattern, '', new_string)

  return new_string

>Read  stop words file 

In [109]:
def read_filestopwords():
    with open('../data/data_stopword/vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
        lines = file.readlines()
        words = [line.split('\n')[0] for line in lines]
    return words

> Remove stop words

In [110]:
def remove_stopword(comment):
  stop_words = read_filestopwords()
  filtered = [word for word in comment.split() if word not in stop_words]
  return ' '.join(filtered)

>Remove repeated words 

In [111]:
def remove_repeated_words(text):
    words = text.split()
    new_words = []
    for i in range(len(words)):
        if i == 0 or words[i] != words[i-1]:
            new_words.append(words[i])
    return ' '.join(new_words)

> Execute function

In [112]:
df_train['comment'] = df_train['comment'].apply(lambda x: x.lower())
df_train['comment'] = df_train['comment'].apply(remove_punctuation)

In [113]:
df_test['comment'] = df_test['comment'].apply(lambda x: x.lower())
df_test['comment']= df_test['comment'].apply(remove_punctuation)

> Tokenizer 

In [114]:
df_train['comment'] = df_train['comment'].apply(lambda x: ViTokenizer.tokenize(x))
df_test['comment']= df_test['comment'].apply(lambda x: ViTokenizer.tokenize(x))

In [115]:
df_train['comment'] = df_train['comment'].apply(remove_stopword)
df_test['comment']= df_test['comment'].apply(remove_stopword)

In [116]:
df_train['comment'].values[95:98]

array(['điện_thoại khá tốt pin trâu khá mượt bắt wifi cực tốt thỉnh_thoảng máy đơ thoát máy ko cập_nhật miul 12',
       'nghe bảo điện_thoại chơi game nóng mua về chiến liên_quân 3 4h liên_tục thấy ấm nóng lời đồn thứ khác vũng chẳng nói quá ngon',
       '1 máy chơi game lướt web ít sử_dụng giữ pin cao nhất khoảng 1 ngày 16 tiếng tính luôn cả ngủ nhé ngủ chẳng động tới 2 ngày mn nói đâu 2 nhạc nghe cà_giật cà_giật khó_chịu luôn 3 quay video mờ chán còn okie'],
      dtype=object)

In [117]:
df_train['comment']=df_train['comment'].apply(remove_repeated_words)
df_test['comment']=df_test['comment'].apply(remove_repeated_words)

In [118]:
df_train['comment'].values[95:98]

array(['điện_thoại khá tốt pin trâu khá mượt bắt wifi cực tốt thỉnh_thoảng máy đơ thoát máy ko cập_nhật miul 12',
       'nghe bảo điện_thoại chơi game nóng mua về chiến liên_quân 3 4h liên_tục thấy ấm nóng lời đồn thứ khác vũng chẳng nói quá ngon',
       '1 máy chơi game lướt web ít sử_dụng giữ pin cao nhất khoảng 1 ngày 16 tiếng tính luôn cả ngủ nhé ngủ chẳng động tới 2 ngày mn nói đâu 2 nhạc nghe cà_giật khó_chịu luôn 3 quay video mờ chán còn okie'],
      dtype=object)

> Observing, we can see that there are 3 types of labels: neutral, positive and negative

In [119]:
df_train.head(5)

,index,comment,n_star,date_time,label
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,{FEATURES#Negative};
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


> Count Features

In [120]:
def count_features(label):
    features = re.findall(r'\{.*?#', label)
    num_features = len(features)
    return num_features

> Execute function

In [121]:
count_features = df_train['label'].apply(count_features)

> Find max features

In [122]:
index_max_features=count_features.idxmax()
number_of_features=df_train.loc[index_max_features,'label']
print(f"Number of features included: {number_of_features}")
print(f"Max features:  {count_features.max()}")

Number of features included: {SCREEN#Positive};{CAMERA#Positive};{FEATURES#Positive};{BATTERY#Positive};{PERFORMANCE#Positive};{DESIGN#Positive};{PRICE#Neutral};{GENERAL#Positive};{SER&ACC#Positive};
Max features:  9


In [123]:
def check_value_features(data):
    for feature in number_of_features:
          if feature in data:
            return True
    return False

>Execute Function

In [124]:
df_train['contains_features'] =df_train['label'].apply(lambda x:check_value_features(x))
print(df_train['contains_features'].count())
# Hiển thị các dòng không chứa ít nhất một trong các tính năng được liệt kê
rows_without_features = df_train[~df_train['contains_features']]
print(rows_without_features)

8898
Empty DataFrame
Columns: [index, comment, n_star, date_time, label, contains_features]
Index: []


> Function count how many characteristics belong to the labels positive, negative, neutral

In [125]:
def count_positive_labels(label):
    return label.count("Positive")
def count_neural_labels(label):
    return label.count("Neutral")
def count_negative_labels(label):
    return label.count("Negative")

> Execute function

In [126]:
df_train['positive_count']=df_train['label'].apply(count_positive_labels)
df_train['neutral_count']=df_train['label'].apply(count_neural_labels)
df_train['negative_count']=df_train['label'].apply(count_negative_labels)
df_test['positive_count']=df_test['label'].apply(count_positive_labels)
df_test['neutral_count']=df_test['label'].apply(count_neural_labels)
df_test['negative_count']=df_test['label'].apply(count_negative_labels)

> Assign label

In [127]:
def assign_label(row):
    if row['positive_count'] > row['neutral_count'] and row['positive_count'] > row['negative_count']:
        return 'Positive'
    elif row['negative_count'] >row['neutral_count'] and row['negative_count'] > row['positive_count']:
        return 'Negative'
    elif row['negative_count'] == row['neutral_count'] :
        return 'Negative'
    elif row['neutral_count']== row ['positive_count']:
        return "Positive"
    else :
        return "Neutral"

>Execute function

In [128]:
df_train['label'] = df_train.apply(assign_label, axis=1)
df_test['label'] = df_test.apply(assign_label,axis=1)

In [129]:
df_train.head(8)

,index,comment,n_star,date_time,label,contains_features,positive_count,neutral_count,negative_count
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,2 tuần trước,Positive,True,6,0,0
1,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,14/09/2019,Neutral,True,1,0,1
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative,True,0,0,1
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral,True,0,2,1
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive,True,2,0,1
5,5,xài tốt mượt pin trâu bạn độ sáng đủ nhân_viên...,5,20/06/2019,Positive,True,3,2,0
6,6,mình mới xài 7 tháng xuống 7 pin chả hiểu máy ...,1,1 tuần trước,Negative,True,0,0,1
7,7,hôm_qua_ngày 2362020 e thế_giới di_động mua dt...,2,23/06/2020,Negative,True,0,0,1


> 

In [130]:
df_train.columns

Index(['index', 'comment', 'n_star', 'date_time', 'label', 'contains_features',
       'positive_count', 'neutral_count', 'negative_count'],
      dtype='object')

In [131]:
df_train.shape

(8898, 9)

>observed that there is asynchronous data in the datetime column

In [132]:
df_train.head(5)

,index,comment,n_star,date_time,label,contains_features,positive_count,neutral_count,negative_count
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,2 tuần trước,Positive,True,6,0,0
1,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,14/09/2019,Neutral,True,1,0,1
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative,True,0,0,1
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral,True,0,2,1
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive,True,2,0,1


> Function Check datetime

In [133]:
def is_valid_date(date_str):
    # Biểu thức chính quy để kiểm tra định dạng ngày tháng
    date_pattern = r'\d{1,2}/\d{1,2}/\d{4}'
    return bool(re.match(date_pattern, date_str))

> Execute function

In [134]:
valid_dates = df_train[df_train['date_time'].apply(is_valid_date)]
valid_dates_test=df_test[df_test['date_time'].apply(is_valid_date)]

> Observing we see that there are 6930 valid values

In [135]:
valid_dates.shape

(7927, 9)

In [136]:
valid_dates.head(5)

,index,comment,n_star,date_time,label,contains_features,positive_count,neutral_count,negative_count
1,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,14/09/2019,Neutral,True,1,0,1
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative,True,0,0,1
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral,True,0,2,1
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive,True,2,0,1
5,5,xài tốt mượt pin trâu bạn độ sáng đủ nhân_viên...,5,20/06/2019,Positive,True,3,2,0


In [137]:
invalid_dates=df_train[~df_train['date_time'].apply(is_valid_date)]
invalid_dates_test=df_test[~df_test['date_time'].apply(is_valid_date)]

> There are 856 values in the datetime column out of a total of 6930 that have inconsistent formatting

In [138]:
invalid_dates.shape

(971, 9)

In [139]:
invalid_dates.head(5)

,index,comment,n_star,date_time,label,contains_features,positive_count,neutral_count,negative_count
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,2 tuần trước,Positive,True,6,0,0
6,6,mình mới xài 7 tháng xuống 7 pin chả hiểu máy ...,1,1 tuần trước,Negative,True,0,0,1
11,11,khá ổn trong tầm giá cam đẹp sạc nhanh màn_hìn...,5,3 tuần trước,Positive,True,4,2,0
14,14,mình mua dc 1 tuần máy phát trực_tiếp tự_nhiên...,1,6 ngày trước,Negative,True,0,0,3
18,18,dùng đơ máy bấm mãi mới khởi_động song nóng ra...,1,5 ngày trước,Negative,True,0,0,1


> Fill in the common value for that attribute

In [140]:
common_value=valid_dates['date_time'].mode()[0]
common_value_test=valid_dates_test['date_time'].mode()[0]
invalid_dates.loc[:, 'date_time'] = common_value
invalid_dates_test.loc[:,'date_time']=common_value

In [141]:
invalid_dates.head(5)

,index,comment,n_star,date_time,label,contains_features,positive_count,neutral_count,negative_count
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,21/04/2020,Positive,True,6,0,0
6,6,mình mới xài 7 tháng xuống 7 pin chả hiểu máy ...,1,21/04/2020,Negative,True,0,0,1
11,11,khá ổn trong tầm giá cam đẹp sạc nhanh màn_hìn...,5,21/04/2020,Positive,True,4,2,0
14,14,mình mua dc 1 tuần máy phát trực_tiếp tự_nhiên...,1,21/04/2020,Negative,True,0,0,3
18,18,dùng đơ máy bấm mãi mới khởi_động song nóng ra...,1,21/04/2020,Negative,True,0,0,1


> combine invalid_dates and valid_dates

In [142]:
df_train=pd.concat([valid_dates,invalid_dates],ignore_index=True)
df_test=pd.concat ([valid_dates_test,invalid_dates_test],ignore_index=True)

In [143]:
df_train.shape

(8898, 9)

In [144]:
df_train.head(5)

,index,comment,n_star,date_time,label,contains_features,positive_count,neutral_count,negative_count
0,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,14/09/2019,Neutral,True,1,0,1
1,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative,True,0,0,1
2,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral,True,0,2,1
3,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive,True,2,0,1
4,5,xài tốt mượt pin trâu bạn độ sáng đủ nhân_viên...,5,20/06/2019,Positive,True,3,2,0


>Remove index and contains_features columns 

In [145]:
df_train.drop(["index",'contains_features'],axis=1,inplace=True)
df_test.drop(['index'],axis=1,inplace=True)

In [146]:
df_train.shape

(8898, 7)

In [147]:
df_train.head(5)

,comment,n_star,date_time,label,positive_count,neutral_count,negative_count
0,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,14/09/2019,Neutral,1,0,1
1,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative,0,0,1
2,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral,0,2,1
3,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive,2,0,1
4,xài tốt mượt pin trâu bạn độ sáng đủ nhân_viên...,5,20/06/2019,Positive,3,2,0


In [148]:
df_train['comment'][0]

'pin kém còn miễn chê mua 832019 tình_trạng pin còn 88 ai giống tôi'

> Dump file CSV after data processing

In [149]:
df_train.to_csv("../data/data_processed/trainprocessed.csv", index=False)
df_test.to_csv("../data/data_processed/testprocesssed.csv",index=False)